In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

sample_submission.csv
test.csv
train.csv



In [6]:
cd ..

/kaggle


In [7]:
cd input

/kaggle/input


# To Do
- Figure out which (open source) Bazyes code is out there that we might want to use. You can learn about Naive Bayes here. There is quite a lot more on the web. You can get some Python-specific examples here.   
    
- Start writing the glue logic.   

- Figure out the format for the (training and test) data and how it may need to be reformatted to suit our needs. The training and test data are here. Incidentally, they are csv (comma-spaced-variables) format. There is Python libraries built specifically for handling csv files, which you can read about here.   

- Start writing code for each of the six Bayes classifiers (toxic, severe_toxic, obscene, threat, insult, identity_hate). Ideally, the will be six different sub-teams allocated to each of the topics. It is envisioned that the six different classifiers will have different feature inputs to the Bayes classifiers, hence each of the six sub-teams can concentrate on their specific topic.   
  
- Integrate the six classifiers into the glue logic to generate the output csv file that is to be uploaded to Kaggle.

In [8]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize

In [9]:
df_train = pd.read_csv('train.csv')
df_train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [13]:
all_words = []
for line in df_train.comment_text.values:
    token_lines = word_tokenize(line)
    for w in token_lines:
        all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())[:2000] # increasing the list size will make the model strong
word_features[:5]

['explanation', 'why', 'the', 'edits', 'made']

In [15]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    count = 5
    for w in word_features:
        features[w] = (w in words)
    return features

train_toxic = [(find_features(line), toxic) for line, toxic in zip(df_train.comment_text.values, df_train.toxic.values)]
train_toxic[:5]

[({'explanation': False,
   'why': False,
   'the': True,
   'edits': True,
   'made': True,
   'under': True,
   'my': True,
   'username': True,
   'hardcore': False,
   'metallica': False,
   'fan': False,
   'were': True,
   'reverted': True,
   '?': True,
   'they': False,
   "n't": True,
   'vandalisms': True,
   ',': True,
   'just': True,
   'closure': True,
   'on': True,
   'some': True,
   'gas': False,
   'after': True,
   'i': False,
   'voted': True,
   'at': True,
   'new': False,
   'york': False,
   'dolls': False,
   'fac': False,
   '.': True,
   'and': False,
   'please': True,
   'do': True,
   'remove': True,
   'template': True,
   'from': True,
   'talk': True,
   'page': True,
   'since': True,
   "'m": True,
   'retired': True,
   'now.89.205.38.27': True,
   "d'aww": False,
   '!': False,
   'he': False,
   'matches': False,
   'this': False,
   'background': False,
   'colour': False,
   'seemingly': False,
   'stuck': False,
   'with': False,
   'thanks': F

In [16]:
classifier = nltk.NaiveBayesClassifier.train(train_toxic)
print("Original Naive Bayes Algo accuracy percent:",
      (nltk.classify.accuracy(classifier, train_toxic))*100)
classifier.show_most_informative_features(50)

Original Naive Bayes Algo accuracy percent: 62.8792199084
Most Informative Features
                 fucking = True                1 : 0      =    166.2 : 1.0
                    fuck = True                1 : 0      =    141.8 : 1.0
                    cock = True                1 : 0      =    140.3 : 1.0
                    dick = True                1 : 0      =     89.9 : 1.0
                 asshole = True                1 : 0      =     83.6 : 1.0
                 sucking = True                1 : 0      =     69.0 : 1.0
                  sucker = True                1 : 0      =     68.1 : 1.0
                  fucked = True                1 : 0      =     51.1 : 1.0
                     ass = True                1 : 0      =     47.7 : 1.0
                    shit = True                1 : 0      =     47.0 : 1.0
                   cunts = True                1 : 0      =     40.9 : 1.0
                      db = True                0 : 1      =     40.2 : 1.0
                

In [21]:
ZeroR_accuracy = 1 - sum(df_train.toxic.values)/len(df_train.toxic.values)
print(ZeroR_accuracy)

0.90415551698


This is the classical bog of words approach where each word is a feature. For more about the method above go here http://www.nltk.org/book/ch06.html
This model performs really bad (so bad that it would perform better if you just guessed all as non toxic), I'm not surprised since kaggle challeges (the ones with prizes) often have a tricky problem. However, there is alot of room for improvement.


# Ideas for how to improve
The above is the basic minimum and not likely to place in kaggle. To improve this, I suggest a better representation such as word2vec embedding: https://www.tensorflow.org/tutorials/word2vec   
This has the improvement ove the bag of words approach since word order and their inability to represent idiomatic phrases.  For example, the meanings of “Canada” and “Air” cannot be easily combined to obtain “Air Canada”. Word2Vec can give us a good vector representations of our input. Training a word2vec requires massive, and better dataset, and more computing power than I can dedicate. However, I suggest embedding our input with a pretrained one. For example: https://code.google.com/archive/p/word2vec/   
This one is quite powerfull, it is pretrained with google news data from a 100 Billion (emphesis on the B) words.

As for the classification, we can use any and all then compaire the results. There is good evidence to suggest that ensable models (combining multiple models) often does better than any of the individual, a long as the all the models within it can perform ok (better than the Zero Rule Algorithm). So we can keep adding models that do relitively well, and at the end take a weighted average. I've also seen a chain of residual models, where model B takes the output of model A and trains it aginst the atual output, then model C takes the output of B as input and trains aginst the actual output, and so on. Each additional model is there to learn the errors of the preceeding one.